# Decision Tree

### import libs

In [1267]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import numpy as np

### load datates

In [1268]:
dataset = load_wine()

x = dataset.data   
y = dataset.target 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2) # 20% test & 80% for train

#### Gini Impurity

1 - &#8721;<sub>i=1</sub><sup>k</sup> (p<sub>i</sub><sup>2</sup>) \
2 - (3n/N)<sup>2</sup> - (2n/N)<sup>2</sup> - (1n/N)<sup>2</sup> - 1 = Gini

In [1269]:
def giniIndex(groups, classes):
    instancesCount = len(groups[0]) + len(groups[1])

    gini = 0.0
    for group in groups:
        size = len(group)
        score = 0.0
        for classVal in classes:
            proportion = (group[:, -1] == classVal).sum() / size if size > 0 else 0
            score += proportion**2
        gini += (1.0 - score) * (size / instancesCount)
    return gini

### split data base on condition

In [1270]:
# split data based on feature column in data set compare to value
def splitData(feature, value, dataset):
    left = dataset[dataset[:, feature] <= value]
    right = dataset[dataset[:, feature] > value]
    return left, right

### build nodes
labels and samples horizontally together \
argsort(): sort array and return indices in unsorted

In [1271]:
def buildNodes(x, y):
    global root, bestFeatureIndex, conditionValue
    dataset = np.hstack(
        (x, y.reshape(-1, 1))
    ) 
    classes = np.unique(y)  # in iris : [0 1 2]
    bestGini = float("inf")  # infinite number fo compare
    for i in range(x.shape[1]):  # shape 1 for every feature do de loop
        sortedIndices = x[
            :, i
        ].argsort()  
        sortedFeature = x[sortedIndices, i]
        pairMeans = (
            (sortedFeature[:-1] + sortedFeature[1:])
            / 2  # remove first and last index, and calc mean
            if len(sortedFeature) > 1
            else [sortedFeature[0]]
        )

        for value in pairMeans:
            left, right = splitData(i, value, dataset) # return more and less of pair mean
            gini = giniIndex([left, right], classes)
            if gini < bestGini:
                bestGini = gini
                bestFeatureIndex = i
                conditionValue = value

    root = bestGini

### build tree
build tree is a recursive function. \
it's base condition is based on depth and class to be uniq (purity)

In [1272]:
def buildTree(x_train, y_train, depth=0, maxDepth=10):
    if (
        depth >= maxDepth or len(np.unique(y_train)) == 1
    ):
        return {"leaf": True, "value": np.bincount(y_train).argmax()}

    buildNodes(x_train, y_train)

    leftIndices = x_train[:, bestFeatureIndex] <= conditionValue
    rightIndices = x_train[:, bestFeatureIndex] > conditionValue

    leftSubtree = buildTree(x_train[leftIndices], y_train[leftIndices], depth + 1)
    rightSubtree = buildTree(x_train[rightIndices], y_train[rightIndices], depth + 1)

    return {
        "leaf": False,
        "feature": bestFeatureIndex,
        "value": conditionValue,
        "left": leftSubtree,
        "right": rightSubtree,
    }


tree = buildTree(x_train, y_train)

### Predict 
Predict is a recursive function. \
it's base condition is based on if tree get to the left

In [1273]:
def predict(sample, tree):
    if tree["leaf"]:
        return tree["value"]
    if sample[tree["feature"]] <= tree["value"]:
        return predict(sample, tree["left"])
    else:
        return predict(sample, tree["right"])


def predictBatch(x_test, tree):
    predictions = []
    for sample in x_test:
        predictions.append(predict(sample, tree))
    return np.array(predictions)

### result

In [1274]:
from sklearn.metrics import  accuracy_score

predictions = predictBatch(x_test, tree)
accuracy = accuracy_score(y_test, predictions)

print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 77.78%
